# Breakout Strategy

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

In [2]:
spy = pd.read_excel('./data/spy_components.xlsx')
ticker = spy['Ticker'].dropna().tolist()
ticker = ' '.join(ticker)
ticker = ticker.replace('.', '-')
data = yf.download(ticker, start="2014-01-01")
# data.to_csv('./data/spy_data.csv')
close = data['Close']
high = data['High']
low = data['Low']

[*********************100%***********************]  504 of 504 completed

1 Failed download:
- CASH_USD: No data found for this date range, symbol may be delisted


## Lookback Range

In [3]:
def get_high_lows_lookback(high, low, lookback_days):
    """
    Get the highs and lows in a lookback window.
    
    Parameters
    ----------
    high : DataFrame
        High price for each ticker and date
    low : DataFrame
        Low price for each ticker and date
    lookback_days : int
        The number of days to look back
    
    Returns
    -------
    lookback_high : DataFrame
        Lookback high price for each ticker and date
    lookback_low : DataFrame
        Lookback low price for each ticker and date
    """
    lookback_high = high.rolling(window = lookback_days).max().shift()
    lookback_low = low.rolling(window = lookback_days).min().shift()

    return lookback_high, lookback_low

In [4]:
lookback_days = 50
lookback_high, lookback_low = get_high_lows_lookback(high, low, lookback_days)

In [8]:
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)  
 
def plot_high_low(stock):
    # create figure
    fig = go.Figure()
    
    # add trace
    fig.add_trace(
        go.Scatter(
            x = close.index,
            y = close[stock],
            line = dict(color="blue", width=2),
            name = "Close"
            )
        )

    fig.add_trace(
    go.Scatter(
        x = lookback_high.index,
        y = lookback_high[stock],
        line = dict(color="green", width=2),
        name = "Lookback High"
        )
    )

    fig.add_trace(
    go.Scatter(
        x = lookback_low.index,
        y = lookback_low[stock],
        line = dict(color="red", width=2),
        name = "Lookback Low"
        )
    )
    
    # update layout
    fig.update_layout(
        title = "Low/ High"
        )
    
    # show figure
    iplot(fig)   

plot_high_low("KO")

## Positions

In [9]:
def get_long_short(close, lookback_high, lookback_low):
    """
    Generate the signals long, short, and do nothing.
    
    Parameters
    ----------
    close : DataFrame
        Close price for each ticker and date
    lookback_high : DataFrame
        Lookback high price for each ticker and date
    lookback_low : DataFrame
        Lookback low price for each ticker and date
    
    Returns
    -------
    long_short : DataFrame
        The long, short, and do nothing signals for each ticker and date
    """
    df_short = lookback_low>close
    df_long = lookback_high<close
    long_short = df_long.astype(int)-df_short.astype(int)
    
    return long_short

In [10]:
signal = get_long_short(close, lookback_high, lookback_low)

In [17]:
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)  

def plot_signals(signal, stock):
    signal_plot = abs(signal*close)

    # create figure
    fig = go.Figure()
    
    # add trace
    fig.add_trace(
        go.Scatter(
            x = close.index,
            y = close[stock],
            line = dict(color="blue", width=2),
            name = "Close"
            )
        )

    # add trace
    fig.add_trace(
        go.Scatter(
            x = close.index,
            y = signal_plot[stock].where(signal[stock] == 1 ),
            mode = 'markers',
            marker =dict(symbol='triangle-up', size = 13, color = 'green'),
            name = "Long Trade"
            )
        )

    # add trace
    fig.add_trace(
        go.Scatter(
            x = close.index,
            y = signal_plot[stock].where(signal[stock] == -1 ),
            mode = 'markers',
            marker =dict(symbol='triangle-down', size = 13, color = 'red'),
            name = "Short Trade"
            )
        )

    
    # update layout
    fig.update_layout(
        title = "Trading Signals"
        )
    
    # show figure
    iplot(fig)   

plot_signals(signal, "KO")

## Filter Signals

In [14]:
def clear_signals(signals, window_size):
    """
    Clear out signals in a Series of just long or short signals.
    
    Remove the number of signals down to 1 within the window size time period.
    
    Parameters
    ----------
    signals : Pandas Series
        The long, short, or do nothing signals
    window_size : int
        The number of days to have a single signal       
    
    Returns
    -------
    signals : Pandas Series
        Signals with the signals removed from the window size
    """
    # Start with buffer of window size
    # This handles the edge case of calculating past_signal in the beginning
    clean_signals = [0]*window_size
    
    for signal_i, current_signal in enumerate(signals):
        # Check if there was a signal in the past window_size of days
        has_past_signal = bool(sum(clean_signals[signal_i:signal_i+window_size]))
        # Use the current signal if there's no past signal, else 0/False
        clean_signals.append(not has_past_signal and current_signal)
        
    # Remove buffer
    clean_signals = clean_signals[window_size:]

    # Return the signals as a Series of Ints
    return pd.Series(np.array(clean_signals).astype(np.int), signals.index)


def filter_signals(signal, holding_period):
    """
    Filter out signals in a DataFrame.
    
    Parameters
    ----------
    signal : DataFrame
        The long, short, and do nothing signals for each ticker and date
    holding_period : int
        The number of days to look ahead
    
    Returns
    -------
    filtered_signal : DataFrame
        The filtered long, short, and do nothing signals for each ticker and date
    """
    df_long = (signal>0).astype(int)
    df_short = (signal<0).astype(int)
    
    for column in signal.columns:
        df_long[column] = clear_signals(df_long[column], holding_period)
        df_short[column] = clear_signals(df_short[column], holding_period)
        
    
    return df_long-df_short

In [18]:
signal_5 = filter_signals(signal, 5)
signal_10 = filter_signals(signal, 10)
signal_20 = filter_signals(signal, 20)

plot_signals(signal_20, "KO")

C:\Users\tbastia\AppData\Local\Temp\ipykernel_10364\2282937446.py:33: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



## Lookahead Close Prices


In [19]:
def get_lookahead_prices(close, lookahead_days):
    """
    Get the lookahead prices for `lookahead_days` number of days.
    
    Parameters
    ----------
    close : DataFrame
        Close price for each ticker and date
    lookahead_days : int
        The number of days to look ahead
    
    Returns
    -------
    lookahead_prices : DataFrame
        The lookahead prices for each ticker and date
    """
    lookahead_prices = close.shift(-lookahead_days)
    return lookahead_prices

In [20]:
lookahead_5 = get_lookahead_prices(close, 5)
lookahead_10 = get_lookahead_prices(close, 10)
lookahead_20 = get_lookahead_prices(close, 20)

## Lookahead Price Returns

In [21]:
def get_return_lookahead(close, lookahead_prices):
    """
    Calculate the log returns from the lookahead days to the signal day.
    
    Parameters
    ----------
    close : DataFrame
        Close price for each ticker and date
    lookahead_prices : DataFrame
        The lookahead prices for each ticker and date
    
    Returns
    -------
    lookahead_returns : DataFrame
        The lookahead log returns for each ticker and date
    """
    lookahead_returns =  np.log(lookahead_prices) - np.log(close)
    return lookahead_returns

In [22]:
price_return_5 = get_return_lookahead(close, lookahead_5)
price_return_10 = get_return_lookahead(close, lookahead_10)
price_return_20 = get_return_lookahead(close, lookahead_20)

def plot_price_return(stock):
    fig = go.Figure()

    # add trace
    fig.add_trace(
        go.Scatter(
            x = price_return_5.index,
            y = price_return_5[stock],
            line = dict(color="blue", width=2),
            name = "Lookahead 5"
            )
        )

    # add trace
    fig.add_trace(
        go.Scatter(
            x = price_return_10.index,
            y = price_return_10[stock],
            line = dict(color="gray", width=2),
            name = "Lookahead 10"
            )
        )

    # add trace
    fig.add_trace(
        go.Scatter(
            x = price_return_20.index,
            y = price_return_20[stock],
            line = dict(color="navy", width=2),
            name = "Lookahead 20"
            )
        )

    # update layout
    fig.update_layout(
        title = "Trading Signals"
        )

    # show figure
    iplot(fig)   

plot_price_return("KO")

## Compute the Signal Return
Using the price returns generate the signal returns.

In [24]:
def get_signal_return(signal, lookahead_returns):
    """
    Compute the signal returns.
    
    Parameters
    ----------
    signal : DataFrame
        The long, short, and do nothing signals for each ticker and date
    lookahead_returns : DataFrame
        The lookahead log returns for each ticker and date
    
    Returns
    -------
    signal_return : DataFrame
        Signal returns for each ticker and date
    """
    signal_return = signal * lookahead_returns
    return signal_return

signal_return_5 = get_signal_return(signal_5, price_return_5)
signal_return_10 = get_signal_return(signal_10, price_return_10)
signal_return_20 = get_signal_return(signal_20, price_return_20)

In [36]:
from plotly.subplots import make_subplots

def plot_signal_return(signal, signal_return, stock):
    signal_plot = abs(signal*close)
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    signal_return = np.exp((signal_return).cumsum())

    # add trace
    fig.add_trace(
        go.Scatter(
            x = close.index,
            y = close[stock],
            line = dict(color="blue", width=2),
            name = "Index"
            ),
            secondary_y=False
        )

    # add trace
    fig.add_trace(
        go.Scatter(
            x = signal_return.index,
            y = signal_return[stock],
            line = dict(color="navy", width=2),
            name = "Cum Log Return Lookahead 20",
            ),
            secondary_y=True
        )

        # add trace
    fig.add_trace(
        go.Scatter(
            x = close.index,
            y = signal_plot[stock].where(signal[stock] == 1 ),
            mode = 'markers',
            marker =dict(symbol='triangle-up', size = 13, color = 'green'),
            name = "Long Trade"
            )
        )

    # add trace
    fig.add_trace(
        go.Scatter(
            x = close.index,
            y = signal_plot[stock].where(signal[stock] == -1 ),
            mode = 'markers',
            marker =dict(symbol='triangle-down', size = 13, color = 'red'),
            name = "Short Trade"
            )
        )

    # update layout
    fig.update_layout(
        title = "Trading Signals"
        )

    # show figure
    iplot(fig)   

plot_signal_return(signal_20, signal_return_20, "KO")